In [1]:
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import pacf
from statsmodels.regression.linear_model import yule_walker
#from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import pandas as pd
import yfinance as yf
AMZN = data = yf.download('CL=F', start ='2000-08-23' , end='2021-04-22')

#WTI = yf.download('CL=F', 
 #                     start='2017-01-01', 
  #                    end='2019-05-01', 
  #                    progress=False)
AMZN['Change %'] = 100* ((WTI['Adj Close'] / WTI['Open']) - 1)
all_data = AMZN[['Adj Close','Open', 'High', 'Low', 'Close', 'Volume', 'Change %']].round(2)
all_data['Change %'].plot()
#print("There are "+ str(WTI[:int(len(WTI) * 0.6546 )].shape[0]) + " observations in the training data")
#print("There are "+ str(WTI[int(len(WTI) * 0.6546 ):].shape[0]) + " observations in the test data")

#AMZN = pd.read_csv("1h_data.csv")
#AMZN["Date"] = AMZN["Unnamed: 0"]
#AMZN.set_index("Date")
#AMZN.drop("Unnamed: 0", axis=1, inplace=True)
#all_data = AMZN[['Adj Close','Open', 'High', 'Low', 'Close', 'Volume']].round(2)
#all_data.head(10)
cut = int(len(all_data)*0.6546)

[*********************100%***********************]  1 of 1 completed


NameError: name 'WTI' is not defined

In [ ]:
all_data['Open'].plot()

In [ ]:
print("There are "+ str(all_data[: cut].shape[0]) + " observations in the training data")
print("There are "+ str(all_data[cut:].shape[0]) + " observations in the test data")

In [ ]:
def ts_train_test(all_data,time_steps,for_periods):
    '''
    input: 
      data: dataframe with dates and price data
    output:
      X_train, y_train: data from 2013/1/1-2018/12/31
      X_test:  data from 2019 -
      sc:      insantiated MinMaxScaler object fit to the training data
    '''
    # create training and test set
    ts_train = all_data[:cut].iloc[:,0:1].values
    ts_test  = all_data[cut:].iloc[:,0:1].values
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)

    # create training data of s samples and t time steps
    X_train = []
    y_train = []
    y_train_stacked = []
    for i in range(time_steps,ts_train_len-1): 
        X_train.append(ts_train[i-time_steps:i,0])
        y_train.append(ts_train[i:i+for_periods,0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshaping X_train for efficient modelling
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

    inputs = pd.concat((all_data["Change %"][:cut], all_data["Change %"][ cut :]),axis=0).values
    inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
    inputs = inputs.reshape(-1,1)

    # Preparing X_test
    X_test = []
    for i in range(time_steps,ts_test_len+time_steps-for_periods):
        X_test.append(inputs[i-time_steps:i,0])
        
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    return X_train, y_train , X_test

X_train, y_train, X_test = ts_train_test(all_data,5,2)
X_train.shape[0],X_train.shape[1]

In [ ]:
# Make the 3-D shape to a data frame so we can see: 
X_train_see = pd.DataFrame(np.reshape(X_train, (X_train.shape[0],X_train.shape[1])))
y_train_see = pd.DataFrame(y_train)
pd.concat([X_train_see,y_train_see],axis=1)

In [ ]:
# Make the 3-D shape to a data frame so we can see: 
X_test_see = pd.DataFrame(np.reshape(X_test, (X_test.shape[0],X_test.shape[1])))
pd.DataFrame(X_test_see)

In [ ]:
print("There are " + str(X_train.shape[0]) + " samples in the training data")
print("There are " + str(X_test.shape[0]) + " samples in the test data")

In [ ]:
def actual_pred_plot(preds):
    actual_pred = pd.DataFrame(columns = ['Change %', 'prediction'])
    name = all_data.iloc[cut].name
    actual_pred['Change %'] = all_data.loc[name:,'Change %'][0:len(preds)]
    actual_pred['prediction'] = preds[:,0]

    from keras.metrics import MeanSquaredError
    m = MeanSquaredError()
    m.update_state(np.array(actual_pred['Change %']),np.array(actual_pred['prediction']))
    
    return (m.result().numpy(), actual_pred.plot() )

In [ ]:
def ts_train_test_normalize(all_data,time_steps,for_periods):
    '''
    input: 
      data: dataframe with dates and price data
    output:
      X_train, y_train: data from 2013/1/1-2018/12/31
      X_test:  data from 2019 -
      sc:      insantiated MinMaxScaler object fit to the training data
    '''
    # create training and test set
    ts_train = all_data[:cut].iloc[:,0:1].values
    ts_test  = all_data[cut:].iloc[:,0:1].values
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)

    # scale the data
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler(feature_range=(0,1))
    ts_train_scaled = sc.fit_transform(ts_train)

    # create training data of s samples and t time steps
    X_train = []
    y_train = []
    y_train_stacked = []
    for i in range(time_steps,ts_train_len-1): 
        X_train.append(ts_train_scaled[i-time_steps:i,0])
        y_train.append(ts_train_scaled[i:i+for_periods,0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshaping X_train for efficient modelling
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

    inputs = pd.concat((all_data["Change %"][: cut], all_data["Change %"][cut :]),axis=0).values
    inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
    inputs = inputs.reshape(-1,1)
    inputs  = sc.transform(inputs)

    # Preparing X_test
    X_test = []
    for i in range(time_steps,ts_test_len+time_steps-for_periods):
        X_test.append(inputs[i-time_steps:i,0])
        
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    return X_train, y_train , X_test, sc

X_train, y_train, X_test, sc = ts_train_test_normalize(all_data,5,2)

In [ ]:
def LSTM_model(X_train, y_train, X_test, sc):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD
    
    # The LSTM architecture
    my_LSTM_model = Sequential()
    my_LSTM_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    my_LSTM_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    my_LSTM_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    my_LSTM_model.add(LSTM(units=50, activation='tanh'))
    my_LSTM_model.add(Dense(units=2))

    # Compiling
    my_LSTM_model.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
    # Fitting to the training set
    my_LSTM_model.fit(X_train,y_train,epochs=50,batch_size=150, verbose=0)

    LSTM_prediction = my_LSTM_model.predict(X_test)
    LSTM_prediction = sc.inverse_transform(LSTM_prediction)

    return my_LSTM_model, LSTM_prediction

my_LSTM_model, LSTM_prediction = LSTM_model(X_train, y_train, X_test, sc)
LSTM_prediction[1:10]
actual_pred_plot(LSTM_prediction)

In [ ]:
def LSTM_model_regularization(X_train, y_train, X_test, sc):

    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, LSTM
    from keras.optimizers import SGD
    from keras.layers import Dropout
    
    # The LSTM architecture
    my_LSTM_model = Sequential()
    # First LSTM layer with Dropout regularisation
    my_LSTM_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
    my_LSTM_model.add(Dropout(0.2))
    # Second LSTM layer
    my_LSTM_model.add(LSTM(units=50, return_sequences=True, activation='tanh'))
    my_LSTM_model.add(Dropout(0.2))
    
    # Third LSTM layer
    my_LSTM_model.add(LSTM(units=50, return_sequences=True, activation='tanh'))
    my_LSTM_model.add(Dropout(0.2))
    # Fourth LSTM layer
    my_LSTM_model.add(LSTM(units=50, activation='tanh'))
    my_LSTM_model.add(Dropout(0.2))
    # The output layer
    my_LSTM_model.add(Dense(units=1))
    # Compiling the RNN
    my_LSTM_model.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
    # Fitting to the training set
    my_LSTM_model.fit(X_train,y_train,epochs=50,batch_size=150, verbose=0)

    LSTM_predictions = my_LSTM_model.predict(X_test)
    LSTM_predictions = sc.inverse_transform(LSTM_predictions)

    return my_LSTM_model, LSTM_predictions

my_LSTM_model, LSTM_predictions = LSTM_model_regularization(X_train, y_train, X_test, sc)
LSTM_predictions[1:10]
actual_pred_plot(LSTM_prediction)


In [ ]:
'''result = LSTM_prediction[:,0]   
    
f= open("1h-pred-lstm.txt","w")

for i in range(0,len(result)):
    f.write(str(result[i]) +",")    

f.close()'''